In [82]:
import pandas as pd
from tqdm import tqdm
import random
import string
import pickle
import spacy
from spacy.util import minibatch
from spacy import displacy

In [83]:
from google.colab import drive
drive.mount('/content/drive')

%cd "drive/MyDrive/Work/Test spacy 2"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/Work/Test spacy 2'
/content/drive/MyDrive/Work/Test spacy 2


In [84]:
test_df = pd.read_csv('test.csv')
test_np = test_df.values
test_df.head()

,id,raw_address
0,0,s. par 53 sidanegara 4 cilacap tengah
1,1,"angg per, baloi indah kel. lubuk baja"
2,2,"asma laun, mand imog,"
3,3,"ud agung rej, raya nga sri wedari karanganyar"
4,4,"cut mutia, 35 baiturrahman"


### preprocess function

In [85]:
def replace_coma(value):
    return value.replace(', ', ' , ')

#### Preprocess coma

In [86]:
# test_df["raw_address"] = test_df["raw_address"].apply(replace_coma)

### Load model

In [101]:
spacy.require_gpu(gpu_id=0)

street_model_path = 'all_model-70/'
nlp = spacy.load(street_model_path)

### Test Predict

In [102]:
sample = test_df.loc[0, "raw_address"]
sample

's. par 53 sidanegara 4 cilacap tengah'

In [103]:
doc = nlp(sample)
# displacy.render(doc, style='ent')
doc.ents

(s. par,)

In [104]:
doc = nlp(sample)
# displacy.render(doc, style='ent')
doc.ents

(s. par,)

In [105]:
[(ent.text, ent.label_) for ent in doc.ents]

[('s. par', 'STREET')]

### Generate submission

In [106]:
output = []
for _id, address in tqdm(test_np):
    doc = nlp(address)
    poi = ''
    street = ''

    for ent in doc.ents:
        label = ent.label_
        if label == 'POI':
            poi = ent.text
        else:
            street = ent.text

    answer = "{}/{}".format(poi.strip(), street.strip())
    output.append({'id': _id, 'POI/street': answer})

100%|██████████| 50000/50000 [03:36<00:00, 230.87it/s]


In [107]:
output_df = pd.DataFrame(output)
output_df.to_csv('sp2-70.csv', index=False)

In [33]:
test_df.head()

,id,raw_address
0,0,s. par 53 sidanegara 4 cilacap tengah
1,1,"angg per, baloi indah kel. lubuk baja"
2,2,"asma laun, mand imog,"
3,3,"ud agung rej, raya nga sri wedari karanganyar"
4,4,"cut mutia, 35 baiturrahman"
